# Grade distribution on Univercity of Copenhagen
**Exam numbers: 32, 37, 119, 212**

**Introduction:**

This python notebook contains a detailed description of the code we used for our exam project.
In the project we describe the gathering process and structuring of the data we gathered from two seperate websites in order to find correlations between grading on KU in relation to different subgroups. We analyse the findings, and put them into perspective of each other and the data logs. Furhtermore, we discuss problems in our methodology, and finally conclude on our results.

This notebook is structured as follows:
1. Scrapeing, logging, and parsing procedures for course data
1. Scrapeing, logging, and parsing procedures for grade data
1. Logging figures
1. Data structureing
1. Merge of the two datasets
1. Results in relation to
    1. Assessment type 
    1. Faculties
    1. Education level
    1. Ordinary and re-exams
    1. Cencorship
    1. Year and Season
    1. Weighing
    

When working on the project, we used various notebooks to develop and improve our code. In this notebook we only show the final code, and figures. Even though this notebook only presents our main findings, it is made to present our project in a fulfilling manner. The notebook gives a explanation of our code and therefore also a detailed understanding of how we came to our results.




## Course scraping, logging and parsing

In [1]:
#Import nessesary modules to run the code 
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
import scraping_class

#Create log file for the process  
logfile = 'log_hand_over.txt'
connector = scraping_class.Connector(logfile)

In [2]:
#Name log file
import scraping_class
logfile = 'log_hand_over.txt'
connector = scraping_class.Connector(logfile)

from bs4 import BeautifulSoup
import requests
main_url = 'https://kurser.ku.dk/archive'
response = connector.get(main_url,'year_archives_links')
raw_html = response[0].text

# Collecting the soup
soup = BeautifulSoup(raw_html,'html.parser')
from tqdm import tqdm_notebook

### UCPH  Courses - Step 1: Collecting links for the yearly archives

In [3]:
# Collecting all the hyperlinks from the mainpage
all_links = [a.get('href') for a in soup.find_all('a', href=True)]

# Saving all the hyperlinks which contains the 'archive'
endpoint_links = [s for s in all_links if "archive" in s]

# Removing 'archive' from the endpoint links since the word is already in the main_url
endpoint_links = [w[7:] for w in endpoint_links]
endpoint_links

['/2017-2018', '/2016-2017', '/2015-2016', '/2014-2015', '/2013-2014']

In [4]:
# Creating the right links to scrape from the yearly archives
desired_links_year = []
for i in range(0,len(endpoint_links)):
    url = main_url + endpoint_links[i]
    desired_links_year.append(url)
desired_links_year

['https://kurser.ku.dk/archive/2017-2018',
 'https://kurser.ku.dk/archive/2016-2017',
 'https://kurser.ku.dk/archive/2015-2016',
 'https://kurser.ku.dk/archive/2014-2015',
 'https://kurser.ku.dk/archive/2013-2014']

### UCPH Courses - Step 2: Collceting links for each Study Board

In [5]:
soup_2 = []
all_links_2 = []
endpoint_links_2 = []
endpoint_links_2_final = []
desired_links_study_board = []

for i in range(0,len(desired_links_year)):
    
    main_url_2 = desired_links_year[i]
    print(main_url_2)
    
    response = connector.get(main_url_2,'year_archives_links')
    raw_html_2 = response[0].text
    # Collecting the soup
    soup_2.append(BeautifulSoup(raw_html_2,'html.parser'))
    # soup_2 is a list of lists, each lists contains html code for each yearly archive.
    
    # Collecting all the hyperlinks from the mainpage
    all_links_2.append([a.get('href') for a in soup_2[i].find_all('a', href=True)])
    
    # Saving the given year, this step only needs to be run once. 
    if i == 0:
        given_year = [w[29:] for w in desired_links_year]
   
    # Saving all the hyperlinks which contains the given year, these are the links for the Study boards
    endpoint_links_2.append([s for s in all_links_2[i] if given_year[i] in s])
    
    # Removing the year from the endpoint_links_2 links since the word is already in the desired_links_year
    endpoint_links_2_final.append([b[9:] for b in endpoint_links_2[i]])
    
    # Creating the right links to scrape from the study board archives
    for k in range(0,len(endpoint_links_2_final[i])): 
        
        # Getting elements in the lists out as strings in the correct way
        str_studyboard = ''.join(endpoint_links_2_final[i][0+k:1+k])
        
        desired_links_study_board.append(desired_links_year[i] + str_studyboard)
        
soup_2;
all_links_2;
given_year;
endpoint_links_2;
endpoint_links_2_final;
desired_links_study_board;

https://kurser.ku.dk/archive/2017-2018
https://kurser.ku.dk/archive/2016-2017
https://kurser.ku.dk/archive/2015-2016
https://kurser.ku.dk/archive/2014-2015
https://kurser.ku.dk/archive/2013-2014


### UCPH Courses - Step 3: Collecting links for each course

In [6]:
desired_links_study_board_rev = []

# We have to edit til disered_links_study_board because the course url do not contain the study board url.
for q in range(0,len(desired_links_study_board)):
    desired_links_study_board_rev.append(desired_links_study_board[q][0:38])

In [7]:
soup_3 = []
all_links_3 = []
endpoint_links_3 = []
endpoint_links_3_final=[]
desired_links_courses = []

#Looping through all studyboards in a given year and collecting the course links
for i in range(0,len(desired_links_study_board)):
    main_url_3 = desired_links_study_board[i]
    print(main_url_3)
    
    response = connector.get(main_url_3,'course_study_board_years_archives_links')
    raw_html_3 = response[0].text
    # Collecting the soup
    soup_3.append(BeautifulSoup(raw_html_3,'html.parser'))
    print(len(soup_3)) 
        
    # Collecting all the hyperlinks from the course page from a given study board in a given year
    all_links_3.append([a.get('href') for a in soup_3[i].find_all('a', href=True)])
    
    # Saving all the hyperlinks which contains the given year, these are the links for the Study boards
    endpoint_links_3.append([s for s in all_links_3[i] if '/course/' in s])
    
    # Removing the year and studyboard from endpoint_links_3 
    endpoint_links_3_final.append([b[18:] for b in endpoint_links_3[i]]) 
    
    # Running through each study board and collecting the course urls.
    for k in range(0,len(endpoint_links_3_final[i])):
        
        # Getting elements in the lists out as strings in the correct way
        # Desired_links_courses is a list containing all course links. 
        desired_links_courses.append(desired_links_study_board_rev[i] + ''.join(endpoint_links_3_final[i][k]))


https://kurser.ku.dk/archive/2017-2018/888
1
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0010
2
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0021
3
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0044
4
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0009
5
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_PB_01
6
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_SU_0031
7
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0012
8
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0033
9
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0025
10
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0019
11
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD__BI_0031
12
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0028
13
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0027
14
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0029
15
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0034
16
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0030
17
https://kurser.ku.dk/archiv

https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_MA_0013
142
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0030
143
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0031
144
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_PHD_0010
145
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0001
146
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0005
147
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_MA_0014
148
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0006
149
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0016
150
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0091
151
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0003
152
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0203
153
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0014
154
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0004
155
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0023
156
https://kurser.ku.dk/archive/2013-2014/STUDYBOARD_0010
157
https://kurser.ku.dk/archive/2013-2014/STUDYBO

In [8]:
print(len(desired_links_courses))
print(len(set(desired_links_courses)))
#we have no dublicates - which makes sence.

20035
20035


### UCPH Courses - Collecting the course ID's

In [9]:
course_ids = []

for i in range(0,len(desired_links_courses)):
    course_ids.append(desired_links_courses[i][46:])


print(len(course_ids))

course_ids[0]

20035


'UCPHAUK18U'

In [10]:
# A set object is an unordered collection of distinct hashable objects - we convert the set back to a list.
unique_course_ids = list(set(course_ids))
len(unique_course_ids)
# We see that we have some dublicates in the course_ids list. This is becuase some courses do not change ID from year to year.

11126

In [11]:
#The first course in the KU archieve is at CBS not KU and is of wierd formatting. We drop the course.
print(course_ids[0])
print(unique_course_ids[0])
print(desired_links_courses[0])

course_ids.remove('UCPHAUK18U')
unique_course_ids.remove('UCPHAUK18U')
desired_links_courses.remove('https://kurser.ku.dk/archive/2017-2018/course/UCPHAUK18U')

UCPHAUK18U
HMKK0323GU
https://kurser.ku.dk/archive/2017-2018/course/UCPHAUK18U


### UCPH Courses - Scraping each course 

In [12]:
#Import modules
import pandas as pd
import numpy as np
import html5lib

In [13]:
def create_df(url):
    # Requesting URL with get
    response = connector.get(url,'course_links')
    
    # Getting course_id and year direct from URL
    course_id = url.split('/')[-1]
    year = url.split('/')[-3]

    # Converting to soup
    raw_html = response[0].text
    soup = BeautifulSoup(raw_html,'lxml')

    # Info used to scrabe certain tables (exam and on in the right corner)
    info = soup.find_all("dl", {'class':'dl-horizontal'})
    
    # Generating error list if course page is out of match
    error = []

    # Only selecting courses with 2 tabels i.e. exactly 1 exam and the info bar to the right
    # Then appending information of each side by 'dt' and 'dd' and pairing them
    if len(info) == 2:
        comp_info = pd.DataFrame()
        cleaned_id_text = ['Course Name']
        for i in info[0].find_all('dt'):
            cleaned_id_text.append(i.text)
        cleaned_id__attrb_text = [soup.find('h1',{'class':"courseTitle"}).text]
        for i in info[0].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)
        
        # If there is a blank space the lists can't be paired. Therfore if lengths dosen't match, list will be created by siblings if sibling isen't on the initial side.
        if len(info[1].find_all('dt')) == len(info[1].find_all('dd')):
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dd'):
                cleaned_id__attrb_text.append(i.text)
        else:
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dt'):
                if i.next_sibling in info[1].find_all('dt'):
                    cleaned_id__attrb_text.append('')
                else:
                    cleaned_id__attrb_text.append(i.next_sibling.text)

        # Appending year from URL
        cleaned_id_text.append('Year')
        cleaned_id__attrb_text.append(year)
        
        # Appending course id form URL
        cleaned_id_text.append('Course id')
        cleaned_id__attrb_text.append(course_id)
        
        # Scrabing data from workload table
        work = soup.find('ul',{'class':'list-unstyled workload clearfix'})
        if work != None:
            workfi = work.find_all('li')
            # Geting 1 list of element, therfore has to be paire 1-3 2-4 etc.
            for i in range(len(workfi)):
                if i % 2 ==0:
                    cleaned_id_text.append(workfi[i].text)
                else: 
                    cleaned_id__attrb_text.append(workfi[i].text)

        # Appending content
        if soup.find_all('div',{'class':"course-content"}) != []:
            cleaned_id_text.append(soup.find_all('div',{'class':"course-content"})[0].find('a').text)
            cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-content"})[0].find('div').text)
        
        # Appending learning outcome
        if soup.find_all('div',{'class':"course-description"}) != []:
            cleaned_id_text.append(soup.find_all('div',{'class':"course-description"})[0].find('a').text)
            cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-description"})[0].find('div').text)

        comp_info['Id'] = cleaned_id_text
        comp_info['Attribute'] = cleaned_id__attrb_text
    else:
        error.append(url)
        comp_info = []
        
    return(comp_info, error)

In [14]:
#Create a list of wierd urls
dct_raw = dict()
wierd_urls = []

for i in tqdm_notebook(range(0,len(desired_links_courses))):
    url = desired_links_courses[i]
    print(desired_links_courses[i])
    dftest,error=create_df(url)
    
    # No need for a column called Id. We just use the Id column as the index. For now.
    # Also, we rename the other column ('Attribute') to the course url since we will use this as the key in the dict. 
    if type(dftest) != list:
        dftest=dftest.set_index('Id').rename(columns = {'Attribute': desired_links_courses[i]})
        
        # Converting the dataframe to a dictionary
        dct_temp = dict()
        dct_temp = dftest.to_dict()
    
        # Adding to dict
        dct_raw.update(dct_temp)
        
    else: 
        wierd_urls.append(error)

https://kurser.ku.dk/archive/2017-2018/course/AANA17100U
https://kurser.ku.dk/archive/2017-2018/course/AANA17101U
https://kurser.ku.dk/archive/2017-2018/course/AANA17102U
https://kurser.ku.dk/archive/2017-2018/course/AANA17103U
https://kurser.ku.dk/archive/2017-2018/course/AANA17104U
https://kurser.ku.dk/archive/2017-2018/course/AANA17105U
https://kurser.ku.dk/archive/2017-2018/course/AANA20003U
https://kurser.ku.dk/archive/2017-2018/course/AANA20004U
https://kurser.ku.dk/archive/2017-2018/course/AANB05023U
https://kurser.ku.dk/archive/2017-2018/course/AANB05024U
https://kurser.ku.dk/archive/2017-2018/course/AANB05044U
https://kurser.ku.dk/archive/2017-2018/course/AANB05059U
https://kurser.ku.dk/archive/2017-2018/course/AANB05081U
https://kurser.ku.dk/archive/2017-2018/course/AANB05090U
https://kurser.ku.dk/archive/2017-2018/course/AANB05091U
https://kurser.ku.dk/archive/2017-2018/course/AANB05093U
https://kurser.ku.dk/archive/2017-2018/course/AANB11002U
https://kurser.ku.dk/archive/20

KeyboardInterrupt: 

In [15]:
dct = dct_raw

In [16]:
# This code translates keys in danish to the english names. 

# This is a list of urls (keys) which is NOT in the wierd_url list.
lst_dct_keys = list(dct.keys())

for i in range(0,len(dct)):

    # Running through all the keys for a given course i and translating keys
    if 'Hjælpemidler' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Aid'] = dct[lst_dct_keys[i]].pop('Hjælpemidler')
        
    if 'Kategori' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Category'] =dct[lst_dct_keys[i]].pop('Kategori')
        
    if 'Censurform' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Censorship form'] =dct[lst_dct_keys[i]].pop('Censurform')
        
    if 'Kursuskode' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Course code'] =dct[lst_dct_keys[i]].pop('Kursuskode')
        
    if 'Point' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Credit'] =dct[lst_dct_keys[i]].pop('Point')
        
    if 'Varighed' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Duration'] =dct[lst_dct_keys[i]].pop('Varighed')
    
    if 'Eksamen' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam'] =dct[lst_dct_keys[i]].pop('Eksamen')
    
    if 'Eksamensperiode' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam period'] =dct[lst_dct_keys[i]].pop('Eksamensperiode')
    
    if 'Sprog' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Language'] =dct[lst_dct_keys[i]].pop('Sprog')
    
    if 'Forelæsninger' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Lectures'] =dct[lst_dct_keys[i]].pop('Forelæsninger')    
        
    if 'Niveau' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Level'] =dct[lst_dct_keys[i]].pop('Niveau')
        
    if 'Bedømmelsesform' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Marking scale'] =dct[lst_dct_keys[i]].pop('Bedømmelsesform')
        
    if 'Placering' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Placement'] =dct[lst_dct_keys[i]].pop('Placering')

    if 'Forberedelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Preparation'] =dct[lst_dct_keys[i]].pop('Forberedelse')
        
    if 'Reeksamen' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Re-exam'] =dct[lst_dct_keys[i]].pop('Reeksamen')
        
    if 'Skemagruppe' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Schedule'] =dct[lst_dct_keys[i]].pop('Skemagruppe')
        
    if 'Studienævn' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Study board'] =dct[lst_dct_keys[i]].pop('Studienævn')
        
    if 'I alt' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Total'] =dct[lst_dct_keys[i]].pop('I alt')

    if 'Prøveform' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Type of assessment'] =dct[lst_dct_keys[i]].pop('Prøveform')
        
    if 'Krav til indstilling til eksamen' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam registration requirements'] =dct[lst_dct_keys[i]].pop('Krav til indstilling til eksamen')
    
    if 'Kursusindhold' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Content'] =dct[lst_dct_keys[i]].pop('Kursusindhold')
    
    if 'Målbeskrivelser' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Learning\r\nOutcome'] =dct[lst_dct_keys[i]].pop('Målbeskrivelser')
        
    if 'Vejledning' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Guidance'] =dct[lst_dct_keys[i]].pop('Vejledning')
        
    if 'Holdundervisning' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Class Exercises'] =dct[lst_dct_keys[i]].pop('Holdundervisning')
        
    if 'Projektarbejde' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Project work'] =dct[lst_dct_keys[i]].pop('Projektarbejde')
        
    if 'Feltarbejde' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Field Work'] =dct[lst_dct_keys[i]].pop('Feltarbejde')
        
    if 'Eksamensforberedelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam Preparation'] =dct[lst_dct_keys[i]].pop('Eksamensforberedelse')
        
    if 'Studiegrupper' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Study Groups'] =dct[lst_dct_keys[i]].pop('Studiegrupper')
        
    if 'Teoretiske øvelser' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Theory exercises'] =dct[lst_dct_keys[i]].pop('Teoretiske øvelser')
        
    if 'Ekskursioner' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Excursions'] =dct[lst_dct_keys[i]].pop('Ekskursioner')
        
    if 'Klinik' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Clinical practice'] =dct[lst_dct_keys[i]].pop('Klinik')
        
    if 'E-læring' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['E-Learning'] =dct[lst_dct_keys[i]].pop('E-læring')
        
    if 'Kollokvier' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Colloquia'] =dct[lst_dct_keys[i]].pop('Kollokvier')

    if 'Laboratorie' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Laboratory'] =dct[lst_dct_keys[i]].pop('Laboratorie')
        
    if 'Praktiske øvelser' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Practical exercises'] =dct[lst_dct_keys[i]].pop('Praktiske øvelser')
        
    if 'Kursuskapacitet' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Course capacity'] =dct[lst_dct_keys[i]].pop('Kursuskapacitet')
        
    if 'Efter- og videreuddannelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Continuing and further education'] =dct[lst_dct_keys[i]].pop('Efter- og videreuddannelse')
        
    if 'Seminarhold' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Class Seminar'] =dct[lst_dct_keys[i]].pop('Seminarhold')
        
    if 'Pris' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Price'] =dct[lst_dct_keys[i]].pop('Pris')
        
    if 'Undervisningsforberedelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Course Preparation'] =dct[lst_dct_keys[i]].pop('Undervisningsforberedelse')

In [17]:
#Import packages
import matplotlib.pyplot as plt
from matplotlib import pyplot
from pylab import *

In [18]:
#Turn results into DataFrame
df = pd.DataFrame.from_dict(dct)
df = df.T
df['URL'] = df.index
index = pd.Series(range(19120))
df = df.set_index(index)

ValueError: Length mismatch: Expected axis has 40 elements, new values have 19120 elements

In [ ]:
for col in df.columns:
    for i in range(1,1+len(df.columns)):
        subplot(2,2,i)
        plt.figure()
        plt.hist(df[col].fillna('').apply(len))
        plt.title(col)

In [ ]:
np.save('kurser_scrabe_ALL.npy', dct) 
np.save('wierd_urls_ALL.npy', wierd_urls)

## 2. Grade scraping, logging and parsing

#### Scrape archive for exam page URL's

In [ ]:
#Initiate the list in which the URL's are to be stored
list_of_grade_urls = [] 

#Define the archive url
url = "http://karakterstatistik.stads.ku.dk/Search/Courses"

#Loop over each link on every archive page using the .post method. Sending a dictionary  
for i in range(95):
    data = {
        "searchText": "", 
        "term": "", 
        "block": "", 
        "faculty": "",
        "institute": "null",
        "page": i,
        }
    response = requests.post(url, data=data)
    html = response.text
    soup = BeautifulSoup(html)
    links = soup.findAll('a')
    list_of_grade_urls.extend([link['href'] for link in links if link.has_attr('href')])

In [ ]:
#Make sure the links are unquoted from ascii to utf-8
list_of_grade_urls
for i in range(len(list_of_grade_urls)):
    list_of_grade_urls[i] = unquote(list_of_grade_urls[i])

In [ ]:
#Define the dataframe 
row = ["URL","Kursus","Fakultet", "Institut","Termin","ECTS","Eksamenstype","Antal tilmeldte",\
        "Fremmødte", "Antal bestået", "Eksamensgennemsnit",12,10,7,4,2,0,-3,\
        "Ej mødt","Ikke bestået"]
df = pd.DataFrame(row)

#Define log file
import scraping_class
logfile = 'log_grade.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

In [ ]:
#Define scraper
errormessages = []
faultyurls = []
urls = list_of_grade_urls[0:10]
for j in tqdm_notebook(range(0,len(urls))):
    
    #url = base_url + sem[j]
    url = urls[j]
    s = connector.get(url,"grade_scrape")
    s_new = s[0].text
    s = BeautifulSoup(s_new)
    x0 = (str(s.findAll("h2"))[7:])[:-6]
    
    #IDENTIFY WEBSITE AND SCRAPE
    try:
        dff = pd.read_html(quote(url, safe=':/'),encoding='utf-8')
    
        #SOFT RESULTS
        keys = []
        values = []

        for i in range(0,len(dff[0][0])):
            keys.append(dff[0][0][i])
            values.append(dff[0][1][i])
        dictionary = dict(zip(keys,values))

        if "Fakultet" in dictionary:
            x1 = dictionary["Fakultet"]
        else:
            x1 = np.nan
        if "Institut" in dictionary:
            x2 = dictionary["Institut"]
        else:
            x2 = np.nan
        if "Termin" in dictionary:
            x3 = dictionary["Termin"]
        else:
            x3 = np.nan
        if "ECTS" in dictionary:
            x4 = dictionary["ECTS"]
        else:
            x4 = np.nan
        if "Ordinær eksamen" in dictionary:
            x5 = dictionary["Ordinær eksamen"]
        else:
            x5 = np.nan
        if "Antal tilmeldte" in dictionary:
            x6 = dictionary["Antal tilmeldte"]
        else:
            x6 = np.nan
        if "Fremmødte" in dictionary:
            x7 = dictionary["Fremmødte"]
        else:
            x7 = np.nan

        #data1 = [x0,x1,x2,x3,x4,x5,x6,x7]
    
        #HARD RESULTS
        if len(dff) >= 4:
            Karakter = []
            Antal = []
            for i in range(0,len(dff[2]["Karakter"])):
                Karakter.append(dff[2]["Karakter"][i])
                Antal.append(dff[2]["Antal"][i])
            dictionary = dict(zip(Karakter,Antal))
            #print(dictionary)
            Beståetlist = []
    
            tolv = 12
            ti = 10
            syv = 7
            fire = 4
            nulto = 2
            nulnul = 0
            minustre = -3

            newkeys = []
            newvalues = []
            if "12" in dictionary:
                newkeys.append(tolv)
                newvalues.append(dictionary["12"])
                Beståetlist.append(dictionary["12"])
                x_12 = dictionary["12"]
            elif 12 in dictionary:
                newkeys.append(tolv)
                newvalues.append(dictionary[12])
                Beståetlist.append(dictionary[12])
                x_12 = dictionary[12]
            else:
                x_12 = np.nan
            if "10" in dictionary:
                newkeys.append(ti)
                newvalues.append(dictionary["10"])
                Beståetlist.append(dictionary["10"])
                x_10 = dictionary["10"]
            elif 10 in dictionary:
                newkeys.append(ti)
                newvalues.append(dictionary[10])
                Beståetlist.append(dictionary[10])
                x_10 = dictionary[10]
            else:
                x_10 = np.nan
            if "7" in dictionary:
                newkeys.append(syv)
                newvalues.append(dictionary["7"])
                Beståetlist.append(dictionary["7"])
                x_7 = dictionary["7"]
            elif 7 in dictionary:
                newkeys.append(syv)
                newvalues.append(dictionary[7])
                Beståetlist.append(dictionary[7])
                x_7 = dictionary[7]
            else:
                x_7 = np.nan
            if "4" in dictionary:
                newkeys.append(fire)
                newvalues.append(dictionary["4"])
                Beståetlist.append(dictionary["4"])
                x_4 = dictionary["4"]
            elif 4 in dictionary:
                newkeys.append(fire)
                newvalues.append(dictionary[4])
                Beståetlist.append(dictionary[4])
                x_4 = dictionary[4]
            else:
                x_4 = np.nan
            if "02" in dictionary:
                newkeys.append(nulto)
                newvalues.append(dictionary["02"])
                Beståetlist.append(dictionary["02"])
                x_02 = dictionary["02"]
            elif 2 in dictionary:
                newkeys.append(nulto)
                newvalues.append(dictionary[2])
                Beståetlist.append(dictionary[2])
                x_02 = dictionary[2]
            else:
                x_02 = np.nan
            if "00" in dictionary:
                newkeys.append(nulnul)
                newvalues.append(dictionary["00"])
                x_00 = dictionary["00"]
            elif 0 in dictionary:
                newkeys.append(nulnul)
                newvalues.append(dictionary[0])
                x_00 = dictionary[0]
            else:
                x_00 = np.nan
            if "-3" in dictionary:
                newkeys.append(minustre)
                newvalues.append(dictionary["-3"]) 
                x_m3 = dictionary["-3"]
            elif -3 in dictionary:
                newkeys.append(minustre)
                newvalues.append(dictionary[-3]) 
                x_m3 = dictionary[-3]
            else:
                x_m3 = np.nan
            if "Ej mødt" in dictionary:
                Ejmødt = dictionary["Ej mødt"]
            else:
                Ejmødt = np.nan
            if "Ikke bestået" in dictionary:
                Ikkebestået = dictionary["Ikke bestået"]
            else:
                Ikkebestået = np.nan
            if "Bestået" in dictionary:
                Bestået = dictionary["Bestået"]
                Beståetlist = [dictionary["Bestået"]]
            else:
                Bestået = np.nan

            newdick = dict(zip(newkeys,newvalues))
            if len(newdick) == 0:
                gennemsnit = np.nan
            else:
                gennemsnit = sum([xx*yy for xx,yy in newdick.items()])/sum(newdick.values())

            x8 = sum(Beståetlist) 
            x9 = gennemsnit
            x10 = x_12
            x11 = x_10
            x12 = x_7
            x13 = x_4
            x14 = x_02
            x15 = x_00
            x16 = x_m3
            x17 = Ejmødt
            x18 = Ikkebestået
    
            data = [url,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18]
            df[j+1] = data
        else:
            a = np.empty(20)
            a[:] = np.nan
            df[j+1] = a
      #######################  
        if len(dff) == 5:
            keys = []
            values = []

            #Soft results without tilmeldte and fremmødte
            for i in range(0,len(dff[0][0])):
                keys.append(dff[0][0][i])
                values.append(dff[0][1][i])
            dictionary = dict(zip(keys,values))
            if "Ordinær eksamen" in dictionary:
                del dictionary["Ordinær eksamen"]
            if "Antal tilmeldte" in dictionary:
                del dictionary["Antal tilmeldte"]
            if "Fremmødte" in dictionary:
                del dictionary["Fremmødte"]
            dictionary["Reeksamen"] = "Reeksamen"

            #if "kursus" in dictionary:
            #    x0 = dictionary["kursus"]
            #else:
            #    x0 = np.nan
            if "Fakultet" in dictionary:
                x1 = dictionary["Fakultet"]
            else:
                x1 = np.nan
            if "Institut" in dictionary:
                x2 = dictionary["Institut"]
            else:
                x2 = np.nan
            if "Termin" in dictionary:
                x3 = dictionary["Termin"]
            else:
                x3 = np.nan
            if "ECTS" in dictionary:
                x4 = dictionary["ECTS"]
            else:
                x4 = np.nan    
            x5 = dictionary["Reeksamen"]

            data1 = [x0,x1,x2,x3,x4,x5]

            #HARD RESULTS
            Karakter = []
            Antal = []
            for i in range(0,len(dff[4]["Karakter"])):
                Karakter.append(dff[4]["Karakter"][i])
                Antal.append(dff[4]["Antal"][i])
            dictionary = dict(zip(Karakter,Antal))
            #print("Dictionary: ", dictionary)
            Beståetlist = []

            tolv = 12
            ti = 10
            syv = 7
            fire = 4
            nulto = 2
            nulnul = 0
            minustre = -3
    
            newkeys = []
            newvalues = []
            if "12" in dictionary:
                newkeys.append(tolv)
                newvalues.append(dictionary["12"])
                Beståetlist.append(dictionary["12"])
                x_12 = dictionary["12"]
            elif 12 in dictionary:
                newkeys.append(tolv)
                newvalues.append(dictionary[12])
                Beståetlist.append(dictionary[12])
                x_12 = dictionary[12]
            else:
                x_12 = np.nan
            if "10" in dictionary:
                newkeys.append(ti)
                newvalues.append(dictionary["10"])
                Beståetlist.append(dictionary["10"])
                x_10 = dictionary["10"]
            elif 10 in dictionary:
                newkeys.append(ti)
                newvalues.append(dictionary[10])
                Beståetlist.append(dictionary[10])
                x_10 = dictionary[10]
            else:
                x_10 = np.nan
            if "7" in dictionary:
                newkeys.append(syv)
                newvalues.append(dictionary["7"])
                Beståetlist.append(dictionary["7"])
                x_7 = dictionary["7"]
            elif 7 in dictionary:
                newkeys.append(syv)
                newvalues.append(dictionary[7])
                Beståetlist.append(dictionary[7])
                x_7 = dictionary[7]
            else:
                x_7 = np.nan
            if "4" in dictionary:
                newkeys.append(fire)
                newvalues.append(dictionary["4"])
                Beståetlist.append(dictionary["4"])
                x_4 = dictionary["4"]
            elif 4 in dictionary:
                newkeys.append(fire)
                newvalues.append(dictionary[4])
                Beståetlist.append(dictionary[4])
                x_4 = dictionary[4]
            else:
                x_4 = np.nan
            if "02" in dictionary:
                newkeys.append(nulto)
                newvalues.append(dictionary["02"])
                Beståetlist.append(dictionary["02"])
                x_02 = dictionary["02"]
            elif 2 in dictionary:
                newkeys.append(nulto)
                newvalues.append(dictionary[2])
                Beståetlist.append(dictionary[2])
                x_02 = dictionary[2]
            else:
                x_02 = np.nan
            if "00" in dictionary:
                newkeys.append(nulnul)
                newvalues.append(dictionary["00"])
                x_00 = dictionary["00"]
            elif 0 in dictionary:
                newkeys.append(nulnul)
                newvalues.append(dictionary[0])
                x_00 = dictionary[0]
            else:
                x_00 = np.nan
            if "-3" in dictionary:
                newkeys.append(minustre)
                newvalues.append(dictionary["-3"]) 
                x_m3 = dictionary["-3"]
            elif -3 in dictionary:
                newkeys.append(minustre)
                newvalues.append(dictionary[-3]) 
                x_m3 = dictionary[-3]
            else:
                x_m3 = np.nan
            if "Ej mødt" in dictionary:
                Ejmødt = dictionary["Ej mødt"]
            else:
                Ejmødt = np.nan
            if "Ikke bestået" in dictionary:
                Ikkebestået = dictionary["Ikke bestået"]
            else:
                Ikkebestået = np.nan
            if "Bestået" in dictionary:
                Bestået = dictionary["Bestået"]
                Beståetlist = [dictionary["Bestået"]]
            else:
                Bestået = np.nan

            newdick = dict(zip(newkeys,newvalues))
            #print("Newdick: ", newdick)
        
            if len(newdick) == 0:
                gennemsnit = np.nan
            else:
                gennemsnit = sum([xx*yy for xx,yy in newdick.items()])/sum(newdick.values())

            x6 = sum(dictionary.values())
            if "Ej mødt" in dictionary:
                x7 = sum(dictionary.values())-dictionary["Ej mødt"]
            else:
                x7 = sum(dictionary.values())

            x8 = sum(Beståetlist) 
            x9 = gennemsnit
            x10 = x_12
            x11 = x_10
            x12 = x_7
            x13 = x_4
            x14 = x_02
            x15 = x_00
            x16 = x_m3
            x17 = Ejmødt
            x18 = Ikkebestået

            data = [url, x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18]
            m = j+1
            stringz = "re "+str(m)
            df[stringz] = data

        else:
            m = j+1
            stringz = "re " + str(m)
            a = np.empty(20)
            a[:] = np.nan
            df[stringz] = a
    except Exception as e:
        m = j+1
        stringz = "re " + str(m)
        a = np.empty(20)
        a[:] = np.nan
        df[stringz] = a
        errormessages.append(e)
        errormessages.append(j)
        faultyurls.append(url)
        faultyurls.append(j)
    
df.to_pickle('RESULTATER_28_AUG_AFTEN.pkl')
import numpy as np
np.savetxt("FAULTYURLS_28_AUG_AFTEN.txt",faultyurls,delimiter=",")
np.savetxt("ERRORMESSAGES_28_AUG_AFTEN.txt",errormessages,delimiter=",")

## Logging

This code presents the method our logging results are made, but does only do it for one of the log files. To replicate the results of the other log file, simply change the log file. Both of them should be converted into comma separated files when converted into csv.

In [19]:
#Import modules
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote, quote
import scraping_class
from tqdm import tqdm_notebook
import requests,os,time
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the log.csv and define the seperator 
log_df = pd.read_csv('log_course.csv', sep=',') # load the ,-separated log 
dt = log_df.t.values[0]

In [ ]:
# Convert the time column 't' to datetime 
log_df['dt'] = log_df.t.apply(datetime.datetime.fromtimestamp)
log_df['dt'] = pd.to_datetime(log_df.t,unit='s')

#### Create response code / time figure (Works as our error checker)

In [ ]:
# Plot the time it took to make the call
plt.figure(figsize=(15,4))
plt.plot(log_df.dt,log_df.response_code)
plt.title("Figure A - Response code / Time", fontsize = 16, weight = "bold")
plt.ylabel('Response code')

#### Create delta t / time figure 

In [ ]:
# Plot the time it took to make the call
plt.figure(figsize=(15,4))
plt.plot(log_df.dt,log_df.delta_t)
plt.title("Figure B - Delta t / Time", fontsize = 16, weight = "bold")
plt.ylabel('Delta t')

#### Create size of response / time figure 

In [ ]:
plt.figure(figsize=(15,4))
plt.ylabel('Size of response')
plt.xlabel('Scraping progress')
plt.title("Figure C - Size of response / Time", fontsize = 16, weight = "bold")
plt.plot(log_df.dt,log_df.response_size)

#### Create size of response / delta t figure

In [ ]:
plt.title("Figure D - Size of response / Delta t", fontsize = 16, weight = "bold")
plt.ylabel('Size of response')
plt.xlabel('Delta t')
plt.scatter(log_df.delta_t,log_df.response_size)

## Scraping output initial analysis